In [ ]:
%pip install sportsreference
from sportsreference.nfl.teams import Teams
from sportsreference.nfl.boxscore import Boxscores
from sportsreference.nfl.boxscore import Boxscore
import numpy as np
import pandas as pd

year = 2012


In [ ]:
def add_each_element(a, b):
  len1 = len(a)
  len2 = len(b)
  if len1 != len2:
    raise Exception('Trying to add arrays of length ' + str(len1) + ' and ' + str(len2))
  result = []
  for i in range(len1):
    result.append(a[i] + b[i])
  return result

def avg_array(a, b):
  len1 = len(a)
  len2 = len(b)
  if len1 != len2:
    raise Exception('Trying to add arrays of length ' + str(len1) + ' and ' + str(len2))
  result = []
  for i in range(len1):
    b_index = (i + 5) % 10
    result.append((a[i] + b[b_index]) / 2)
  return result


In [ ]:
#Create large Data Store
cumulative_large_dict = {}
for i in range(1, 18):
  print(cumulative_large_dict)
  cumulative_large_dict[i] = {}
  print('doing week ' + str(i))
  weekly_games = Boxscores(i, year)
  game_dict = weekly_games.games
  date_string = str(i) + '-' + str(year)
  box_score_arr = game_dict[date_string]
  for nfl_game in box_score_arr:
    box_score_id = nfl_game['boxscore']
    box_score = Boxscore(box_score_id)
    
    home_team = box_score.home_abbreviation
    stat_array_home = [box_score.home_points, box_score.home_pass_yards, box_score.home_rush_yards, box_score.home_fumbles, box_score.home_interceptions]
  
    away_team = box_score.away_abbreviation
    stat_array_away = [box_score.away_points, box_score.away_pass_yards, box_score.away_rush_yards, box_score.away_fumbles, box_score.away_interceptions]

    full_stat_team_home = stat_array_home + stat_array_away
    full_stat_team_away = stat_array_away + stat_array_home

    winner = box_score.winning_abbr
    if winner == home_team:
      full_stat_team_home.append(1)
      full_stat_team_home.append(0)
      full_stat_team_away.append(0)
      full_stat_team_away.append(1)
    else:
      full_stat_team_home.append(0)
      full_stat_team_home.append(1)
      full_stat_team_away.append(1)
      full_stat_team_away.append(0)

    #Save the data
    if i == 1:
      cumulative_large_dict[i][home_team] = full_stat_team_home
      cumulative_large_dict[i][away_team] = full_stat_team_away
    else:
      if home_team in cumulative_large_dict[i-1]:
        cumulative_large_dict[i][home_team] = add_each_element(cumulative_large_dict[i-1][home_team], full_stat_team_home)
      else:
        print('trying to get ' + home_team)
        cumulative_large_dict[i][home_team] = add_each_element(cumulative_large_dict[i-2][home_team], full_stat_team_home)
      
      if away_team in cumulative_large_dict[i-1]:
        cumulative_large_dict[i][away_team] = add_each_element(cumulative_large_dict[i-1][away_team], full_stat_team_away)
      else:
        cumulative_large_dict[i][away_team] = add_each_element(cumulative_large_dict[i-2][away_team], full_stat_team_away)







In [ ]:
#For each week of games
examples = []
for k in range(2,17):
  weekly_games = Boxscores(k, year)
  game_dict = weekly_games.games
  date_string = str(k) + '-' + str(year)
  box_score_arr = game_dict[date_string]
  for game in box_score_arr:
    home_team = game['home_abbr']
    away_team = game['away_abbr']

    
    #Avg stats home
    if home_team in cumulative_large_dict[k - 1]:
      home_stats = cumulative_large_dict[k - 1][home_team]
    else:
      home_stats = cumulative_large_dict[k - 2][home_team]
    
    total_games_home = home_stats[-1] + home_stats[-2]
    divisor = float(total_games_home)
    home_avg = [home_stats[i] / divisor for i in range(10)]

    #Avg stats away
    if away_team in cumulative_large_dict[k - 1]:
      away_stats = cumulative_large_dict[k - 1][away_team]
    else:
      away_stats = cumulative_large_dict[k - 2][away_team]
    
    total_games_away = away_stats[-1] + away_stats[-2]
    divisor = float(total_games_away)
    away_avg = [away_stats[i] / divisor for i in range(10)]

    #Construct final feature array
    feature = avg_array(home_avg, away_avg)
    home_win_pct = float(home_stats[-1]) / float((home_stats[-2] + home_stats[-1]))
    away_win_pct = float(away_stats[-1]) / float((away_stats[-2] + away_stats[-1]))
    home_field_advantage = away_win_pct - home_win_pct
    feature.append(home_field_advantage)

    #Add target
    home_score = game['home_score']
    away_score = game['away_score']
    #Loss for home team
    target = 0 
    if home_score > away_score:
      #win for home team
      target = 1
    elif home_score == away_score:
      target = [0.5, 0.5]
      print('TIE')
      print(k)
      print(home_team + ' vs. ' + away_team)
    feature.append(home_team)
    feature.append(away_team)
    feature.append(k)
    feature.append(target)
    examples.append(feature)

TIE
10
sfo vs. ram


In [ ]:
cols = ['Home Scoring', 'Home Pass Yds', 'Home Rush Yds', 'Home Fumbles', 'Home Ints', 'Away Scoring', 'Away Pass Yds', 'Away Rush Yds', 'Away Fumbles', 'Away Ints', 'Home Field Adv', 'Home Team', 'Away Team', 'Week', 'Target']
examples_data_frame = pd.DataFrame(np.array(examples), columns=cols)
examples_data_frame

,Home Scoring,Home Pass Yds,Home Rush Yds,Home Fumbles,Home Ints,Away Scoring,Away Pass Yds,Away Rush Yds,Away Fumbles,Away Ints,Home Field Adv,Home Team,Away Team,Week,Target
0,21.5,306.0,54.0,1.5,2.0,35.5,272.0,150.0,0.5,0.5,0.0,gnb,chi,2,1
1,34.0,247.0,139.5,1.0,1.5,36.0,262.0,135.0,1.5,1.5,1.0,buf,kan,2,1
2,25.0,194.5,138.5,1.0,0.5,16.5,243.5,31.5,1.5,1.0,1.0,nwe,crd,2,0
3,16.0,225.0,54.0,1.5,1.5,22.0,281.5,64.0,2.0,0.0,1.0,mia,rai,2,1
4,15.0,269.0,139.5,2.5,2.5,30.0,223.5,110.5,1.5,2.0,1.0,cin,cle,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,24.92857142857143,254.85714285714286,122.07142857142857,1.2142857142857144,0.7142857142857143,20.785714285714285,260.57142857142856,103.89285714285714,1.3928571428571428,0.8214285714285714,0.0714285714285714,car,rai,16,1
220,25.67857142857143,274.96428571428567,115.39285714285714,1.5,0.9285714285714286,19.785714285714285,235.96428571428572,94.21428571428572,1.4285714285714286,1.1785714285714284,0.14285714285714285,den,cle,16,1
221,16.57142857142857,218.07142857142856,94.5,1.7857142857142858,1.3928571428571428,22.25,211.0,128.03571428571428,1.0714285714285714,1.3571428571428572,0.0714285714285714,crd,chi,16,0
222,23.285714285714285,260.1071428571429,114.39285714285714,1.3571428571428572,1.0714285714285714,24.285714285714285,257.17857142857144,123.60714285714286,1.25,1.0,0.0,rav,nyg,16,1


In [ ]:
from google.colab import files
examples_data_frame.to_csv('2012_examples_teams.csv')
files.download('2012_examples_teams.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>